During the process I had real problems with realizing a model that has any valid coherence metric and that yields good results. <br>
My belief is that the dataset I generated contains too much noise to correctly capture the relations we are interested in.

#### In order to measure the quality of the dataset we always try the same configuration on different pre-processing approaches. <br> The most promising will be considered from now on.

Ensure that our dataset might be of bad quality by seeing if the model behaves well for the restaurant dataset used in paper that presented ABAE:

In [1]:
import pandas as pd

ds_path = "../data/restaurant/train.csv"

In [2]:
from core.train import ABAEModelConfiguration, ABAEModelManager
from core.dataset import PandasPositiveNegativeNumericTextDataset

config = ABAEModelConfiguration(corpus_file=ds_path, model_name=f"restaurant")
manager = ABAEModelManager(config)

Pandas Apply:   0%|          | 0/279885 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/279885 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\restaurant\embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\restaurant\\embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\restaurant\embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\restaurant\\embeddings.model', 'datetime': '2025-02-03T17:18:17.108433', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


Loading the existing found model as requested in path ./output/restaurant/embeddings.model
Loading the existing found model as requested in path ./output/restaurant/aspect-embeddings.model


In [ ]:
train_dataset = PandasPositiveNegativeNumericTextDataset(
    dataframe=pd.read_csv(ds_path),
    vocabulary=manager.embedding_model.vocabulary(),
    negative_size=config.negative_sample_size,
    max_seq_length=config.max_sequence_length
)

In [ ]:
_, iteration_model = manager.run_train_process(train_dataset, optimizer='adam')

In [3]:
from core.dataset import PandasNumericTextDataset
import torch
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration
from keras import ops as K

iteration_model = manager.get_training_model()
word_emb = iteration_model.get_layer('word_embeddings').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embeddings').w

word_emb = torch.nn.functional.normalize(word_emb, p=2, dim=-1)
aspect_embeddings = torch.nn.functional.normalize(aspect_embeddings, p=2, dim=-1)

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=50) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = pd.read_csv(ds_path)["comments"].apply(lambda x: x.split(' ')).to_list()
dictionary = corpora.Dictionary(ds)

_, m = coherence_model_generation(aspect_words, ds, dictionary, topn=3)

m.get_coherence()

D:\PycharmProjects\nlp-course-project\core\evaluation.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  similarity = word_embeddings @ aspect.T



Given aspect most representative words are:
Word:  55 (0.5848684906959534)
Word:  23 (0.5821459293365479)
Word:  65 (0.5805290341377258)
Word:  tax (0.5465304255485535)
Word:  incl (0.5459697246551514)
Word:  pp (0.5439687967300415)
Word:  approximately (0.5431109666824341)
Word:  12 (0.5417876243591309)
Word:  costing (0.5355474948883057)
Word:  26 (0.5331826210021973)
Word:  coke (0.5325402617454529)
Word:  130 (0.5296655297279358)
Word:  35 (0.5253255367279053)
Word:  48 (0.5249996185302734)
Word:  75 (0.5245991945266724)
Word:  39 (0.5212913751602173)
Word:  180 (0.5210379362106323)
Word:  28 (0.5169060230255127)
Word:  110 (0.515718936920166)
Word:  whopping (0.5125362873077393)
Word:  300 (0.5108407139778137)
Word:  250 (0.5095862746238708)
Word:  95 (0.5093751549720764)
Word:  99 (0.507595956325531)
Word:  38 (0.5072352886199951)
Word:  125 (0.5065628290176392)
Word:  oz (0.5061278939247131)
Word:  14 (0.5030094385147095)
Word:  pitcher (0.49953097105026245)
Word:  120 (0.49913

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<8179 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<11846 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<14671 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<16894 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<19103 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<20891 unique tokens: ['jeollado', 'like', 'menu', 'price', 'roll']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Dictionary<22657 unique t

-0.1347967269663061

In [7]:
from core.evaluation import coherence_per_aspect

aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
c, m = coherence_per_aspect(aspect_words, pd.read_csv(ds_path)["comments"], 5)

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

In [9]:
c

[-16.109532862673355,
 -16.71302323217839,
 -9.371089059361529,
 -14.921386894040555,
 -16.591165628698956,
 -19.157866318139224,
 -11.046515602791146,
 -11.156699702048162,
 -8.270143637418316,
 -18.30157446952101,
 -11.106924467730721,
 -15.806476106154395,
 -12.106892495930342,
 -11.473925877021532]

In [8]:
m.get_coherence()

-13.723801168121977

In [45]:
# We measured a coherence of: -0.1347967269663061
# Better than the models I trained but still low! Check on the test dataset.
labels = [
    "Misc",
    "Food",
    "Staff",
    "Ambience",
    "Food",
    "Misc",
    "Misc",
    "Ambience",
    "Staff",
    "Staff",
    "Misc",
    "Food",
    "Food",
    "Staff"
]
## Aspect mapping:
# 0 - Money ?
# 1 - Food
# 2 - Ambience
# 3 - Atmosphere
# 4 - Food (Fish)
# 5 - Restaurants
# 6 - Time / Event
# 7 - Ambience
# 8 - Service
# 9 - Staff (People)
# 10 - ?? Place?
# 11 - Food
# 12 - Stile fusion etc
# 13 - Service

In [59]:
m.get_coherence_per_topic()

[-0.38323168409104064,
 0.07741761439495207,
 -0.08134020571322033,
 0.05182736802652027,
 -0.24400517613281025,
 -0.3892263920110865,
 -0.024317270468537062,
 -0.15803965987047108,
 0.18446760588121244,
 -0.35095524500650477,
 -0.09110677569596747,
 -0.15315754404002282,
 -0.17796221815129762,
 -0.14752459465001194]

In [27]:
from torch.utils.data import DataLoader
from core.dataset import SimpleWord2VecEmbeddingsDataset

iteration_model = manager.get_inference_model()

true_labels = pd.read_csv("../data/restaurant/test_label.txt")["Label"]
custom_review = pd.read_csv("../data/restaurant/test.txt")["review"].to_list()

# todo rebuild a prediciton model senza negative predict. Devo ristrutturare tutto1
ds = SimpleWord2VecEmbeddingsDataset(custom_review, manager.embedding_model.model, max_seq_length=80)

res = iteration_model.predict(DataLoader(ds, batch_size=32))

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [35]:
import numpy as np

predicted = np.argmax(res[1], axis=1)

In [33]:
true_labels

0          Staff
1       Ambience
2          Staff
3          Staff
4          Staff
          ...   
1485        Food
1486        Food
1487        Food
1488        Food
1489        Food
Name: Label, Length: 1490, dtype: object

In [46]:
correct = 0
for pred, true in zip(predicted, true_labels):

    print(f"{pred} ({labels[pred]}) : {true}")
    if labels[pred] == true:
        correct += 1
# Meh results. 54%
correct / len(predicted)

8 (Staff) : Staff
9 (Staff) : Ambience
9 (Staff) : Staff
9 (Staff) : Staff
13 (Staff) : Staff
9 (Staff) : Staff
13 (Staff) : Staff
9 (Staff) : Ambience
9 (Staff) : Staff
12 (Food) : Food
1 (Food) : Food
13 (Staff) : Staff
13 (Staff) : Staff
11 (Food) : Food
9 (Staff) : Ambience
8 (Staff) : Staff
12 (Food) : Food
10 (Misc) : Food
9 (Staff) : Food
9 (Staff) : Ambience
9 (Staff) : Ambience
12 (Food) : Food
10 (Misc) : Ambience
12 (Food) : Food
3 (Ambience) : Ambience
9 (Staff) : Food
10 (Misc) : Ambience
8 (Staff) : Staff
1 (Food) : Food
0 (Misc) : Food
12 (Food) : Food
7 (Ambience) : Ambience
9 (Staff) : Ambience
13 (Staff) : Staff
11 (Food) : Food
0 (Misc) : Food
11 (Food) : Food
12 (Food) : Food
13 (Staff) : Ambience
4 (Food) : Food
4 (Food) : Food
8 (Staff) : Staff
12 (Food) : Food
9 (Staff) : Staff
10 (Misc) : Ambience
8 (Staff) : Food
11 (Food) : Food
11 (Food) : Food
10 (Misc) : Food
11 (Food) : Food
2 (Staff) : Ambience
8 (Staff) : Staff
2 (Staff) : Food
12 (Food) : Food
9 (Staff)

0.6852348993288591

In [57]:
for tag in ["Staff", "Food", "Ambience"]:
    correct = 0
    for pred, true in zip(predicted, true_labels):
        if true == tag:
            # print(f"{pred} ({labels[pred]}) : {true}")
            if labels[pred] == true:
                correct += 1
    # Meh results. 54%
    print(correct / (true_labels == tag).sum())

0.8778409090909091
0.7102593010146562
0.32669322709163345


## Most Naive Approach:
Is the default pipeline.

In [51]:
len(true_labels)

1490

## Approach 2:
- Be stricter
- Split reviews in longer sequences and not sentences

## Approach 3: